<a href="https://colab.research.google.com/github/BillYerkes/CSC5542/blob/master/Lab2/Spark_ICP_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-bin-hadoop3.2.tgz
!pip install -q findspark

In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!ls /usr/lib/jvm/

default-java		   java-11-openjdk-amd64     java-8-openjdk-amd64
java-1.11.0-openjdk-amd64  java-1.8.0-openjdk-amd64


In [6]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [7]:
!pip install pyspark


     |████████████████████████████████| 204.7MB 29kB/s 
     |████████████████████████████████| 204kB 28.9MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044184 sha256=e7aeb3adab60b3b377e98c450bc7497306cc931c53f00701cb1f051d8a0e0fda
  Stored in directory: /root/.cache/pip/wheels/57/27/4d/ddacf7143f8d5b76c45c61ee2e43d9f8492fc5a8e78ebd7d37
Successfully built pyspark


Configuring a SparkSession

The entry point to using Spark SQL is an object called SparkSession. It initiates a Spark Application which all the code for that Session will run on.

.builder — gives access to Builder API which is used to configure the session .

.master() — determines where the program will run; "local[*]" sets it to run locally on all cores but you can use "local[1]" to run on one core for example. In this case, our programs will be run on Google’s servers.

.appName() — optional method to name the Spark Application

.getOrCreate() — gets an existing SparkSession or creates new one if none exists

In [10]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("Big_Data_Application").getOrCreate()

To open a local file on Google Colab you need to run the following code which will prompt you to select a file from your computer:

In [31]:
from google.colab import files
files.upload()

Saving icp2.txt to icp2.txt


{'icp2.txt': b'The University of South Carolina reports that more than 1,000 students currently have the virus.\r\nThe C.D.C. tells health officials to be ready to distribute a vaccine by November, raising concerns over politicized timing.\r\nIn Iowa, college students staged a sickout, and a football opener won\xe2\x80\x99t have fans after all.\r\nVirus fallout from the Sturgis motorcycle rally: A death in Minnesota, cases in South Dakota and more.\r\nNew studies show inexpensive steroid drugs can help critically sick people survive Covid-19.\r\nSilvio Berlusconi, Italy\xe2\x80\x99s former prime minister, tests positive.\r\nA judge orders the University of California to stop considering SAT or ACT scores because of the pandemic.'}

Load the data from the text file and split the data into "word", splitting on spaces.

In [93]:
sc = spark.sparkContext

words = sc.textFile('icp2.txt').flatMap(lambda line: line.split(" "))


Map the words by their starting Letter


In [94]:
xMap = words.map(lambda x: (x[0], x))

Reduce the Map, grouping words together, which start with the same first Letter.

In [95]:
xReduce = xMap.reduceByKey(lambda a,b: a + ' ' + b).collect()

Display the Contents of the Map and Reduce

In [96]:
xMapping = xMap.collect()
print("Map: ")
print(xMapping)
print(" ")
print("Reduce: ")
print(xReduce)

Map: 
[('T', 'The'), ('U', 'University'), ('o', 'of'), ('S', 'South'), ('C', 'Carolina'), ('r', 'reports'), ('t', 'that'), ('m', 'more'), ('t', 'than'), ('1', '1,000'), ('s', 'students'), ('c', 'currently'), ('h', 'have'), ('t', 'the'), ('v', 'virus.'), ('T', 'The'), ('C', 'C.D.C.'), ('t', 'tells'), ('h', 'health'), ('o', 'officials'), ('t', 'to'), ('b', 'be'), ('r', 'ready'), ('t', 'to'), ('d', 'distribute'), ('a', 'a'), ('v', 'vaccine'), ('b', 'by'), ('N', 'November,'), ('r', 'raising'), ('c', 'concerns'), ('o', 'over'), ('p', 'politicized'), ('t', 'timing.'), ('I', 'In'), ('I', 'Iowa,'), ('c', 'college'), ('s', 'students'), ('s', 'staged'), ('a', 'a'), ('s', 'sickout,'), ('a', 'and'), ('a', 'a'), ('f', 'football'), ('o', 'opener'), ('w', 'won’t'), ('h', 'have'), ('f', 'fans'), ('a', 'after'), ('a', 'all.'), ('V', 'Virus'), ('f', 'fallout'), ('f', 'from'), ('t', 'the'), ('S', 'Sturgis'), ('m', 'motorcycle'), ('r', 'rally:'), ('A', 'A'), ('d', 'death'), ('i', 'in'), ('M', 'Minnesota,'

Generate Output for the Lab

In [97]:

#punctuation to remove from the end of words
punctuation = ['.', ':', ',']

#Output file to print results to
with open('output.txt','w') as f:

#iterate over Reduce content
  for x in range( len(xReduce) ):

    #Print the Key
    f.write(xReduce[x][0])
    print(xReduce[x][0], end = '')

    #Print a comma
    f.write(', ')
    print(', ', end = '')

    #Get the words for the Key
    xAllWords = xReduce[x][1]
    xWords = xAllWords.split()
    
    #Loop throug the words, put a comma between words
    #but not at the end
    y = 0

    #Loop to the second to last element
    while (y < (len(xWords) - 1)):

      #If word ends with a puncuation remove it
      if (xWords[y][-1] in puncuation):
        xWords[y] = xWords[y][:-1]

      #Print the word
      f.write(xWords[y])
      print(xWords[y], end = '')

      #print a comma
      f.write(', ')
      print(', ', end = '')
      y = y + 1

    #check the word for key if it ends with a punctuation
    if (xWords[y][-1] in punctuation):
      xWords[y] = xWords[y][:-1]
    
    #print the last element
    f.write(xWords[y])
    f.write('\n')
    print(xWords[y])



S, South, Sturgis, South, Silvio, SAT
C, Carolina, C.D.C, Covid-19, California
r, reports, ready, raising, rally
1, 1,000
s, students, students, staged, sickout, studies, show, steroid, sick, survive, stop, scores
c, currently, concerns, college, cases, can, critically, considering
h, have, health, have, help
b, be, by, because
d, distribute, death, drugs
N, November, New
p, politicized, people, prime, positive, pandemic
i, in, in, inexpensive
j, judge
T, The, The
U, University, University
o, of, officials, over, opener, orders, of, or, of
t, that, than, the, tells, to, to, timing, the, tests, the, to, the
m, more, motorcycle, more, minister
v, virus, vaccine
a, a, a, and, a, after, all, and
I, In, Iowa, Italy’s
f, football, fans, fallout, from, former
w, won’t
V, Virus
A, A, A, ACT
M, Minnesota
D, Dakota
B, Berlusconi
